In [61]:
# Don't care. It's just a command for gist update.
run(`gist -u https://gist.github.com/2e8eb8bb6f86c614458a9adaa675ad93 slides.ipynb`)

https://gist.github.com/2e8eb8bb6f86c614458a9adaa675ad93


$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits}
\newcommand{\argmin}{\mathop{\rm arg~min}\limits}
$$

# Chapter5. 系列ラベリング
啓太

You can access this notebook in [here](http://nbviewer.jupyter.org/gist/KeitaW/2e8eb8bb6f86c614458a9adaa675ad93).

## Glossary
* 系列(Sequence): Nurture passes nature.
* Token: Narture, passes, nature
* 品詞(tag): 名詞(Noun), 動詞(Verb), ...
* __系列ラベリング__(__Sequence Labeling__): Assign appropriate labels to given sequences. <- Objective!

## What we will learn...
* HMM(Hidden Markov Model): Supervised 
    - How to estimate parameters of the model.
    - How to assign labels to a sequence with the trained model (Viterbi algorithm).
* CRF(Conditional Random Field): Supervised
    - How to estimate parameters of the model. (Forward-backward algorithm)
    
    

# Preparation

In [2]:
using PyCall
using DataFrames
using ScikitLearn
using Base.Test
using Base.Iterators

In [3]:
@pyimport nltk.corpus as corpus
function load_tagged_dataset(;ndata=Int(1e+06))
    tagged_words = corpus.brown[:tagged_words]()
    sentences = []; tags_list = []
    words = []; tags = []
    for (word, tag) in tagged_words
        if word == "."
            # Separate sentence by "."
            push!(sentences, words); push!(tags_list, tags)
            words = []; tags = []
        else
            push!(words, lowercase.(word)); push!(tags, tag)
        end
    end
    return DataFrame(
        sentences = sentences,
        tags_list = tags_list)
end

load_tagged_dataset (generic function with 1 method)

In [4]:
tagged_words = load_tagged_dataset()
head(tagged_words)

,sentences,tags_list
1,"Any[""the"", ""fulton"", ""county"", ""grand"", ""jury"", ""said"", ""friday"", ""an"", ""investigation"", ""of"", ""atlanta's"", ""recent"", ""primary"", ""election"", ""produced"", ""``"", ""no"", ""evidence"", ""''"", ""that"", ""any"", ""irregularities"", ""took"", ""place""]","Any[""AT"", ""NP-TL"", ""NN-TL"", ""JJ-TL"", ""NN-TL"", ""VBD"", ""NR"", ""AT"", ""NN"", ""IN"", ""NP\$"", ""JJ"", ""NN"", ""NN"", ""VBD"", ""``"", ""AT"", ""NN"", ""''"", ""CS"", ""DTI"", ""NNS"", ""VBD"", ""NN""]"
2,"Any[""the"", ""jury"", ""further"", ""said"", ""in"", ""term-end"", ""presentments"", ""that"", ""the"", ""city"", ""executive"", ""committee"", "","", ""which"", ""had"", ""over-all"", ""charge"", ""of"", ""the"", ""election"", "","", ""``"", ""deserves"", ""the"", ""praise"", ""and"", ""thanks"", ""of"", ""the"", ""city"", ""of"", ""atlanta"", ""''"", ""for"", ""the"", ""manner"", ""in"", ""which"", ""the"", ""election"", ""was"", ""conducted""]","Any[""AT"", ""NN"", ""RBR"", ""VBD"", ""IN"", ""NN"", ""NNS"", ""CS"", ""AT"", ""NN-TL"", ""JJ-TL"", ""NN-TL"", "","", ""WDT"", ""HVD"", ""JJ"", ""NN"", ""IN"", ""AT"", ""NN"", "","", ""``"", ""VBZ"", ""AT"", ""NN"", ""CC"", ""NNS"", ""IN"", ""AT"", ""NN-TL"", ""IN-TL"", ""NP-TL"", ""''"", ""IN"", ""AT"", ""NN"", ""IN"", ""WDT"", ""AT"", ""NN"", ""BEDZ"", ""VBN""]"
3,"Any[""the"", ""september-october"", ""term"", ""jury"", ""had"", ""been"", ""charged"", ""by"", ""fulton"", ""superior"", ""court"", ""judge"", ""durwood"", ""pye"", ""to"", ""investigate"", ""reports"", ""of"", ""possible"", ""``"", ""irregularities"", ""''"", ""in"", ""the"", ""hard-fought"", ""primary"", ""which"", ""was"", ""won"", ""by"", ""mayor-nominate"", ""ivan"", ""allen"", ""jr.""]","Any[""AT"", ""NP"", ""NN"", ""NN"", ""HVD"", ""BEN"", ""VBN"", ""IN"", ""NP-TL"", ""JJ-TL"", ""NN-TL"", ""NN-TL"", ""NP"", ""NP"", ""TO"", ""VB"", ""NNS"", ""IN"", ""JJ"", ""``"", ""NNS"", ""''"", ""IN"", ""AT"", ""JJ"", ""NN"", ""WDT"", ""BEDZ"", ""VBN"", ""IN"", ""NN-TL"", ""NP"", ""NP"", ""NP""]"
4,"Any[""``"", ""only"", ""a"", ""relative"", ""handful"", ""of"", ""such"", ""reports"", ""was"", ""received"", ""''"", "","", ""the"", ""jury"", ""said"", "","", ""``"", ""considering"", ""the"", ""widespread"", ""interest"", ""in"", ""the"", ""election"", "","", ""the"", ""number"", ""of"", ""voters"", ""and"", ""the"", ""size"", ""of"", ""this"", ""city"", ""''""]","Any[""``"", ""RB"", ""AT"", ""JJ"", ""NN"", ""IN"", ""JJ"", ""NNS"", ""BEDZ"", ""VBN"", ""''"", "","", ""AT"", ""NN"", ""VBD"", "","", ""``"", ""IN"", ""AT"", ""JJ"", ""NN"", ""IN"", ""AT"", ""NN"", "","", ""AT"", ""NN"", ""IN"", ""NNS"", ""CC"", ""AT"", ""NN"", ""IN"", ""DT"", ""NN"", ""''""]"
5,"Any[""the"", ""jury"", ""said"", ""it"", ""did"", ""find"", ""that"", ""many"", ""of"", ""georgia's"", ""registration"", ""and"", ""election"", ""laws"", ""``"", ""are"", ""outmoded"", ""or"", ""inadequate"", ""and"", ""often"", ""ambiguous"", ""''""]","Any[""AT"", ""NN"", ""VBD"", ""PPS"", ""DOD"", ""VB"", ""CS"", ""AP"", ""IN"", ""NP\$"", ""NN"", ""CC"", ""NN"", ""NNS"", ""``"", ""BER"", ""JJ"", ""CC"", ""JJ"", ""CC"", ""RB"", ""JJ"", ""''""]"
6,"Any[""it"", ""recommended"", ""that"", ""fulton"", ""legislators"", ""act"", ""``"", ""to"", ""have"", ""these"", ""laws"", ""studied"", ""and"", ""revised"", ""to"", ""the"", ""end"", ""of"", ""modernizing"", ""and"", ""improving"", ""them"", ""''""]","Any[""PPS"", ""VBD"", ""CS"", ""NP"", ""NNS"", ""VB"", ""``"", ""TO"", ""HV"", ""DTS"", ""NNS"", ""VBN"", ""CC"", ""VBN"", ""IN"", ""AT"", ""NN"", ""IN"", ""VBG"", ""CC"", ""VBG"", ""PPO"", ""''""]"


Following command lists all tags(品詞) and words in this dataset.

In [37]:
uniq_tags = unique(flatten(tagged_words[:tags_list]))
uniq_tags[1:10]

10-element Array{String,1}:
 "AT"   
 "NP-TL"
 "NN-TL"
 "JJ-TL"
 "VBD"  
 "NR"   
 "NN"   
 "IN"   
 "NP\$" 
 "JJ"   

In [6]:
uniq_words = unique(flatten(tagged_words[:sentences]))
uniq_words[1:10]

10-element Array{String,1}:
 "the"          
 "fulton"       
 "county"       
 "grand"        
 "jury"         
 "said"         
 "friday"       
 "an"           
 "investigation"
 "of"           

Let's see meaning of each tag...

In [34]:
@pyimport nltk.help as nlhelp
nlhelp.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

^^;

In [39]:
numdata = Int(1e+03)
tagged_words = tagged_words[1:numdata, :]
X_train, X_test, y_train, y_test = ScikitLearn.CrossValidation.train_test_split(tagged_words[:sentences], tagged_words[:tags_list], test_size = 0.3);
X_train_, X_test_, y_train_, y_test_ =  collect(flatten(X_train)), collect(flatten(X_test)), collect(flatten(y_train)), collect(flatten(y_test));
X_train_[1:10]

10-element Array{String,1}:
 "nothing"  
 "has"      
 "been"     
 "done"     
 "yet"      
 "to"       
 "take"     
 "advantage"
 "of"       
 "the"      

# HMM

## Notation
* $\mathbf{x}$: Data (In this context, it's words).  
* $\mathbf{y}$: Label (In this context, it's tags).

## Assumption
![](http://iacs-courses.seas.harvard.edu/courses/am207/blog/hmm.png)
http://iacs-courses.seas.harvard.edu/courses/am207/blog/lecture-18.html

We may rewrite conditional probability of $\mathbf{x}$ and $\mathbf{y}$ as follows:
$$
\begin{eqnarray}
P(\mathbf{x}, \mathbf{y}) &=&  P(x_{1:T}, y_{1:T}) \\
    &=& P(x_T|x_{1:T-1}, y_{1:T})P(x_{1:T-1}, y_{1:T}) \\
    &=& P(x_T|x_{1:T-1}, y_{1:T})P(y_T | x_{1:T-1}, y_{1:{T-1}})P(x_{1:T-1}, y_{1:T-1}) \\
    &=& \dots \\
    &=& \prod_{i=1}^T  P(x_i|x_{1:i-1}, y_{1:i})P(y_i | x_{1:i-1}, y_{1:i-1}) \quad \text{Use the assumption}\\
    &=& \prod_{i=1}^T P(x_i|y_i)P(y_i|y_{i-1})
\end{eqnarray}
$$
Where $T$ is the number of tokens in $\mathbf{x}$. Note: $P(x_1, y_1) = P(x_1, y_1 | x_0, y_0),  P(y_1) = P(y_1|y_0), P(x_1) = P(x_1|x_0)$

## Parameter estimation of the HMM
Let $D = \{ (\mathbf{x}^{(1)}, \mathbf{y}^{(1)}), (\mathbf{x}^{(2)}, \mathbf{y}^{(2)}), \dots, (\mathbf{x}^{(T)}, \mathbf{y}^{(T)})\}$ be training dataset. We may define log-likelihood:

$$
\begin{align}
\log P(D) &=& \sum_{(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \in D} \log P(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \\
    &=&\sum_{(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \in D} \log \prod_{j=1}^T P(x^{(i)}_j|y^{(i)}_j)P(y^{(i)}_j|y^{(i)}_{j-1}) \\
    &=& \sum_{(\mathbf{x}^{(i)}, \mathbf{y}^{(i)}) \in D} \left( \sum_{j=1}^T \log P(x^{(i)}_j|y^{(i)}_j) + \sum_{j=1}^T  \log P(y^{(i)}_j|y^{(i)}_{j-1}) \right) \\
    &=& \sum_{x, y} n((x, y), D) \log p_{x|y} + \sum_{y, y'} n((y', y), D) \log q_{y|y'}
\end{align}
$$
where $n((x, y), D)$ be number of word $x$ that have tag $y$, $n((y', y), D)$ be number of tag $y$ that follows $y'$, $p_{x|y}=P(x|y)$ and $q_{y|y'}=P(y|y')$.

We want to find $p_{x|y}, q_{y|y'}$ such that
$$
\argmax_{p_{x|y}, q_{y|y'}} \sum_{x, y} n((x, y), D) \log p_{x|y} + \sum_{y, y'} n((y', y), D) \log q_{y|y'}
$$
subject to $\sum_{x}p_{x|y} = 1$ and $\sum_{y}q_{y|y'} = 1$. 
We may use the method of Lagrange multipliers,
$$
\mathscr{L}(p_{x|y}, q_{y|y'}, \alpha_1, \alpha_2) = \sum_{x, y} n((x, y), D) \log p_{x|y} + \sum_{y, y'} n((y', y), D) \log q_{y|y'} + \alpha_1 (1-\sum_x p_{x|y}) + \alpha_2 (1-\sum_y q_{y|y'})
$$
Solving $\partial \mathscr{L} / \partial p_{x|y} = 0,\ \partial \mathscr{L} / \partial q_{y|y'} = 0,\ \partial \mathscr{L} / \partial \alpha_1 = 0 \text{ and } \partial \mathscr{L} / \partial \alpha_2 = 0$ yields

$$
p_{x|y} = \frac{n((x, y), D)}{\sum_x n((x, y), D)}, \quad q_{y|y'} = \frac{n((y', y), D)}{\sum_y n((y', y), D)}
$$

In [49]:
func = (x) -> x^2
func(4)

16

In [41]:
nxy_ = (words, tags) -> word -> tag -> sum((words .== word) .& (tags .== tag))
# n_{tag1 | tag2} in other words, y = tag1; y' = tag2
nyy_ = (words, tags) -> tag1 -> tag2 -> sum((tags .== tag1) .& (circshift(tags, 1) .== tag2))
# Note order of argments flipped 
pxy_ = nxy -> uniq_words -> function (tag)
    denominator = reduce(+, 0, map(w -> nxy(w)(tag), uniq_words))
    (word) -> nxy(word)(tag) == 0 ? 0 : nxy(word)(tag) / denominator
end
pyy_ = nyy -> uniq_tags  -> function (tag2)
    denominator = reduce(+, 0, map(t -> nyy(t)(tag2), uniq_tags))
    (tag1) -> nyy(tag1)(tag2) == 0 ? 0 : nyy(tag1)(tag2) / denominator
end
pxy  = pxy_(nxy_(X_train, y_train))(uniq_words); pyy = pyy_(nyy_(X_train, y_train))(uniq_tags)

(::#130) (generic function with 1 method)

Such code often produces many bugs, write test.

In [10]:
# Test data
twords = ["i", "am", "ai", "i", "am", "governor"]; tuniq_words = unique(twords)
ttags = ["AT", "BT", "AT", "DT", "BT", "AT"]; tuniq_tags = unique(ttags)
@test nxy_(twords, ttags)("i")("AT") == 1
@test nxy_(twords, ttags)("am")("BT") == 2
@test nyy_(twords, ttags)("BT")("AT") == 1
@test nyy_(twords, ttags)("AT")("BT") == 2

Test Passed


In [11]:
tpxy = pxy_(nxy_(twords, ttags))(tuniq_words); tpyy = pyy_(nyy_(twords, ttags))(tuniq_tags) 
@test tpxy("BT")("am") == 1
@test tpyy("BT")("AT") == 1.0
@test tpyy("AT")("BT") == 0.3333333333333333
@test tpyy("DT")("AT") == 0

Test Passed


## Inference with a trained HMM model
We want to acquire optimal estimate of tag assignments $\mathbf{y}^{\max}$ such that satisfy,
$$
\begin{align}
\mathbf{y}^{\max} &=& \argmax_\mathbf{y} \sum_{j} P(\mathbf{x}, \mathbf{y}) = \argmax_\mathbf{y} \sum_{j} \log P(x_j, y_j | x_{j-1}, y_{j-1}) \\
    &=& \argmax_\mathbf{y} \sum_{j} \log P(x_j|y_j) + \log P(y_j|y_{j-1}) \\
    &=& \argmax_\mathbf{y} \sum_{j} \log p_{x_j|y_j} + \log q_{y_j|y_{j-1}} \\
\end{align}
$$

Define $\mathrm{trans}(x_j, y_j, y_{j-1})$ as follows,
$$
\mathrm{trans}(x_j, y_j, y_{j-1}) \equiv \log P(x_j, y_j | x_{j-1}, y_{j-1}) = \log p_{x_j|y_j} + \log q_{y_j|y_{j-1}}
$$

In the end we get,
$$
\mathbf{y}^{\max} = \argmax_\mathbf{y} \sum_j \mathrm{trans}(x_j, y_j, y_{j-1})
$$

### The Viterbi Algorithm
> The Viterbi Algorithm is a dynamic programming algorithm for finding the most likely sequence of hidden states – called the Viterbi path – that results in a sequence of observed events, especially in the context of Markov information sources and hidden Markov models (from Wikipedia).

__objective__: find $\mathbf{y}$ such that satisfy, $\mathbf{y}^{\max} = \argmax_\mathbf{y} \sum_j \mathrm{trans}(x_j, y_j, y_{j-1})$.

This algorithm consists of two parts which will be discussed in the following slides.

$$
\begin{align}
\begin{cases}
t(j, y_j) = \max_{y_{j-1}} [\mathrm{trans}(x_j, y_j, y_{j-1}) + t(j-1, y_{j-1})] \\
s(j, y_j) = \argmax_{y_{j-1}} [\mathrm{trans}(x_j, y_j, y_{j-1}) + t(j-1, y_{j-1})]
\end{cases}
\Leftrightarrow
\begin{cases}
s(j, y_j) = \argmax_{y_{j-1}} [\mathrm{trans}(x_j, y_j, y_{j-1}) + t(j-1, y_{j-1})] \\
t(j, y_j) = \mathrm{trans}(x_j, y_j, s(j, y_j)) + t(j-1, s(j, y_j))
\end{cases}
\end{align} 
$$

In [50]:
# tag: a current tag, ptag: a previous tag
trans_ = (pxy) -> (pyy) -> (word) -> (tag) -> (ptag) -> log(pxy(tag)(word)) + log(pyy(ptag)(tag))
# # equivalent efficient yet bit complex form
# trans_ = (pxy) -> (pyy) -> function (tag)
#     px_ = pxy(tag)
#     function (ptag)
#         second = log(pyy(ptag)(tag))
#         (word) -> log(px_(word)) + second
#     end
# end
@test trans_(tpxy)(tpyy)("i")("AT")("BT") == -1.0986122886681098

Test Passed


In [13]:
viterbi_fit(words, tags) = pxy_(nxy_(words, tags))(unique(words)), pyy_(nyy_(words, tags))(unique(tags))   

viterbi_fit (generic function with 1 method)

In [14]:
@inline function viterbi_forward(pxy, pyy, words, uniq_tags)
    trans = trans_(pxy)(pyy)
    T = zeros(Float64, length(words), length(uniq_tags)); S = zeros(Int64, length(words), length(uniq_tags))
    for j in 1:length(uniq_tags) T[1, j] = log(pxy(uniq_tags[j])(words[1])) end
    for i in 2:length(words)
        for j in 1:length(uniq_tags)
            (T[i, j], S[i, j]) = findmax([trans(words[i])(uniq_tags[j])(uniq_tags[k])+T[i-1, k] for k in 1:length(uniq_tags)])
        end
    end
    return S, T
end

viterbi_forward (generic function with 1 method)

In [15]:
@inline function viterbi_backword(T, S, uniq_tags, words)
    _, j = findmax(T[end, :])
    optimal_tags = []
    push!(optimal_tags, uniq_tags[j])
    for i in length(words):-1:2
        push!(optimal_tags, uniq_tags[S[i, j]])
        j = S[i, j]
    end
    return flipdim(optimal_tags, 1)
end

viterbi_backword (generic function with 1 method)

In [16]:
twords = ["i", "am", "ai", "i", "am", "governor"]; tuniq_words = unique(twords)
ttags = ["AT", "BT", "CT", "DT", "ET", "FT"]; tuniq_tags = unique(ttags)
pxy, pyy = viterbi_fit(twords, ttags)
S, T = viterbi_forward(pxy, pyy, twords, unique(ttags))  

([0 0 … 0 0; 1 1 … 4 1; … ; 1 1 … 4 1; 1 1 … 1 5], [0.0 -Inf … -Inf -Inf; -Inf 0.0 … 0.0 -Inf; … ; -Inf -Inf … 0.0 -Inf; -Inf -Inf … -Inf 0.0])

In [17]:
T

6×6 Array{Float64,2}:
    0.0  -Inf    -Inf       0.0  -Inf    -Inf  
 -Inf       0.0  -Inf    -Inf       0.0  -Inf  
 -Inf    -Inf       0.0  -Inf    -Inf    -Inf  
 -Inf    -Inf    -Inf       0.0  -Inf    -Inf  
 -Inf    -Inf    -Inf    -Inf       0.0  -Inf  
 -Inf    -Inf    -Inf    -Inf    -Inf       0.0

In [18]:
S

6×6 Array{Int64,2}:
 0  0  0  0  0  0
 1  1  1  1  4  1
 1  1  2  1  1  1
 1  1  1  3  1  1
 1  1  1  1  4  1
 1  1  1  1  1  5

In [19]:
optimal_tags = viterbi_backword(T, S, unique(ttags), twords)

6-element Array{Any,1}:
 "AT"
 "BT"
 "CT"
 "DT"
 "ET"
 "FT"

Try the simplest example...

In [53]:
from, til = 1, 250
xtrain = X_train_[from:til]; ytrain = y_train_[from:til]
pxy, pyy = viterbi_fit(xtrain, ytrain)
uniq_tags = unique(ytrain);

In [54]:
from, til = 1, 10
xtest = X_train_[from:til]; ytest = y_train_[from:til];

In [22]:
@show xtrain;
S, T = viterbi_forward(pxy, pyy, xtest, uniq_tags)

xtrain = String["mills", "shot", "out", "in", "front", "and", "kept", "the", "lead", "through", "two", "thirds", "of", "the", "race", "(", "2", ")", "fulton", "legislators", "``", "work", "with", "city", "officials", "to", "pass", "enabling", "legislation", "that", "will", "permit", "the", "establishment", "of", "a", "fair", "and", "equitable", "''", "pension", "plan", "for", "city", "employes", "``", "you", "take", "out", "of", "circulation", "many", "millions", "of", "dollars", "''", "``", "he", "is", "wrong", "to", "inject", "eisenhower", "into", "this", "campaign", "''", ",", "he", "said", ",", "``", "because", "the", "primary", "is", "being", "waged", "on", "state", "issues", "and", "i", "will", "not", "be", "forced", "into", "re-arguing", "an", "old", "national", "campaign", "''", "austin", ",", "texas", "--", "state", "representatives", "decided", "thursday", "against", "taking", "a", "poll", "on", "what", "kind", "of", "taxes", "texans", "would", "prefer", "to", "pay", "trenton

([0 0 … 0 0; 1 1 … 1 1; … ; 1 1 … 1 1; 1 1 … 1 1], [-1.79176 -Inf … -Inf -Inf; -Inf -5.19296 … -Inf -Inf; … ; -Inf -Inf … -Inf -Inf; -Inf -Inf … -Inf -Inf])

In [23]:
S, T = viterbi_forward(pxy, pyy,xtest, uniq_tags);

In [24]:
optimal_tags = viterbi_backword(T, S, uniq_tags, xtest)
@show optimal_tags;
@show ytest;

optimal_tags = Any["NP", "VBD", "RP", "IN", "NN", "CC", "VBD", "AT", "NN", "IN"]
ytest = String["NP", "VBD", "RP", "IN", "NN", "CC", "VBD", "AT", "NN", "IN"]


So far, so good. Try another example.

In [55]:
@show xtest
xtest2 = xtest; xtest2[1] = "???"
@show xtest2;

xtest = String["nothing", "has", "been", "done", "yet", "to", "take", "advantage", "of", "the"]
xtest2 = String["???", "has", "been", "done", "yet", "to", "take", "advantage", "of", "the"]


In [57]:
S, T = viterbi_forward(pxy, pyy, xtest2, uniq_tags)
optimal_tags = viterbi_backword(T, S, uniq_tags, xtest2)
@show optimal_tags;
@show ytest;

optimal_tags = Any["PN", "PN", "PN", "PN", "PN", "PN", "PN", "PN", "PN", "PN"]
ytest = String["PN", "HVZ", "BEN", "VBN", "RB", "TO", "VB", "NN", "IN", "AT"]


In [58]:
T

10×53 Array{Float64,2}:
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf  …  -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf  …  -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf
 -Inf  -Inf  -Inf  -Inf  -Inf  -Inf     -Inf  -Inf  -Inf  -Inf  -Inf  -Inf

<img src="http://i0.kym-cdn.com/entries/icons/original/000/002/252/NoMeGusta.jpg" alt="" width="500px" height="500px">
http://i0.kym-cdn.com/entries/icons/original/000/002/252/NoMeGusta.jpg

We may use possible (messy) workaround which is shown below,

In [27]:
nxy__ = (words, tags) -> word -> tag -> sum((words .== word) .& (tags .== tag)) + 1
# n_{tag1 | tag2} in other words, y = tag1; y' = tag2
nyy__ = (words, tags) -> tag1 -> tag2 -> sum((tags .== tag1) .& (circshift(tags, 1) .== tag2)) + 1

(::#58) (generic function with 1 method)

In [28]:
viterbi_fit_alt(words, tags) = pxy_(nxy__(words, tags))(unique(words)), pyy_(nyy__(words, tags))(unique(tags))

viterbi_fit_alt (generic function with 1 method)

In [59]:
pxy, pyy = viterbi_fit_alt(xtrain, ytrain);
S, T = viterbi_forward(pxy, pyy, xtest2, uniq_tags)
optimal_tags = viterbi_backword(T, S, uniq_tags, xtest2)
@show optimal_tags;
@show ytest;

optimal_tags = Any["JJ", "NN", "IN", "AT", "NN", "IN", "AT", "NN", "IN", "AT"]
ytest = String["PN", "HVZ", "BEN", "VBN", "RB", "TO", "VB", "NN", "IN", "AT"]


In [60]:
T

10×53 Array{Float64,2}:
  -5.09375   -5.09987   -5.09375  …   -5.09987   -5.09375   -5.09375
 -13.5078   -12.3908   -13.5078      -13.6534   -14.0596   -13.9517 
 -21.4919   -21.199    -20.1056      -20.909    -21.1745   -20.9926 
 -28.8599   -28.5555   -28.8599      -28.8349   -28.7057   -28.1667 
 -35.9748   -35.9119   -35.9748      -35.9809   -35.8207   -35.2817 
 -43.3411   -43.2684   -43.3411   …  -43.3472   -42.9948   -42.8128 
 -48.7294   -48.7356   -48.7294      -48.7356   -48.7294   -48.7294 
 -55.7156   -55.7217   -55.7156      -55.7217   -55.7156   -55.0224 
 -62.7355   -62.7416   -62.7355      -62.7416   -62.0424   -62.7355 
 -67.9106   -67.9167   -67.9106      -67.9167   -67.9106   -67.9106 

Good

In [30]:
testdata = ["contributing", "to", "its", "defeat", "the", "new", "year", "might", "see", "some", "house-cleaning"]
testlabel = ["VBG", "IN", "PP\$", "NN", "AT", "JJ", "NN", "MD", "VB", "DTI", "NN"]
S, T = viterbi_forward(pxy, pyy, testdata, uniq_tags)
optimal_tags = viterbi_backword(T, S, uniq_tags, testdata)
@show optimal_tags;
@show testlabel;

optimal_tags = Any["IN", "AT", "NN", "IN", "AT", "NN", "IN", "AT", "NN", "IN", "AT"]
testlabel = String["VBG", "IN", "PP\$", "NN", "AT", "JJ", "NN", "MD", "VB", "DTI", "NN"]


In [31]:
testdata = ["???", "to", "its", "defeat", "the", "new", "year", "might", "see", "some", "house-cleaning"]
testlabel = ["VBG", "IN", "PP\$", "NN", "AT", "JJ", "NN", "MD", "VB", "DTI", "NN"]
S, T = viterbi_forward(pxy, pyy, testdata, uniq_tags)
optimal_tags = viterbi_backword(T, S, uniq_tags, testdata)
@show optimal_tags;
@show testlabel;

optimal_tags = Any["IN", "AT", "NN", "IN", "AT", "NN", "IN", "AT", "NN", "IN", "AT"]
testlabel = String["VBG", "IN", "PP\$", "NN", "AT", "JJ", "NN", "MD", "VB", "DTI", "NN"]
